In [1]:
import numpy as np
import pandas as pd
import function
from unidecode import unidecode
from pandas.api.types import is_string_dtype

In [42]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

In [8]:
# Step 1 : Read the dataset
df = pd.read_csv('Datasets/CompleteDataset.csv', low_memory=False)
df.head()

,Sr.No,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,...,RB,RCB,RCM,RDM,RF,RM,RS,RW,RWB,ST
0,0,Cristiano Ronaldo,32,https://cdn.sofifa.org/48/18/players/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Real Madrid CF,https://cdn.sofifa.org/24/18/teams/243.png,...,61.0,53.0,82.0,62.0,91.0,89.0,92.0,91.0,66.0,92.0
1,1,L. Messi,30,https://cdn.sofifa.org/48/18/players/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,93,93,FC Barcelona,https://cdn.sofifa.org/24/18/teams/241.png,...,57.0,45.0,84.0,59.0,92.0,90.0,88.0,91.0,62.0,88.0
2,2,Neymar,25,https://cdn.sofifa.org/48/18/players/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,94,Paris Saint-Germain,https://cdn.sofifa.org/24/18/teams/73.png,...,59.0,46.0,79.0,59.0,88.0,87.0,84.0,89.0,64.0,84.0
3,3,L. Suárez,30,https://cdn.sofifa.org/48/18/players/176580.png,Uruguay,https://cdn.sofifa.org/flags/60.png,92,92,FC Barcelona,https://cdn.sofifa.org/24/18/teams/241.png,...,64.0,58.0,80.0,65.0,88.0,85.0,88.0,87.0,68.0,88.0
4,4,M. Neuer,31,https://cdn.sofifa.org/48/18/players/167495.png,Germany,https://cdn.sofifa.org/flags/21.png,92,92,FC Bayern Munich,https://cdn.sofifa.org/24/18/teams/21.png,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Step 2 : Drop unnecessary columns
df=df.drop(['Sr.No','Photo','Flag','Club Logo','Special'],axis=1)

In [16]:
# Step 3 : Convert accented characters to normal form.

df['Name'] = df['Name'].apply(lambda x:unidecode(str(x)))
df['Club'] = df['Club'].apply(lambda x:unidecode(str(x)))

In [20]:
nan_cols = df.isna().sum()      # get columns which contain NaN values along with count
nan_cols = nan_cols.to_dict()
nan_cols = [key for key, value in nan_cols.items() if value!=0]
nan_cols

['CAM',
 'CB',
 'CDM',
 'CF',
 'CM',
 'LAM',
 'LB',
 'LCB',
 'LCM',
 'LDM',
 'LF',
 'LM',
 'LS',
 'LW',
 'LWB',
 'RAM',
 'RB',
 'RCB',
 'RCM',
 'RDM',
 'RF',
 'RM',
 'RS',
 'RW',
 'RWB',
 'ST']

In [22]:
for col in nan_cols:
    df[col] = df[col].fillna(df[col].mean())

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17981 entries, 0 to 17980
Data columns (total 72 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Name                 17981 non-null  object 
 1   Age                  17981 non-null  int64  
 2   Nationality          17981 non-null  object 
 3   Overall              17981 non-null  int64  
 4   Potential            17981 non-null  int64  
 5   Club                 17981 non-null  object 
 6   Value                17981 non-null  object 
 7   Wage                 17981 non-null  object 
 8   Height_cm            17981 non-null  int64  
 9   Weight_kg            17981 non-null  int64  
 10  Acceleration         17981 non-null  object 
 11  Aggression           17981 non-null  object 
 12  Agility              17981 non-null  object 
 13  Balance              17981 non-null  object 
 14  Ball control         17981 non-null  object 
 15  Composure            17981 non-null 

In [26]:
# step 5 : make each observations in each column uniform and convert into appropriate data type

attrs=['Acceleration', 'Aggression', 'Agility', 'Balance', 'Ball control', 'Composure',
   'Crossing', 'Curve', 'Dribbling', 'Finishing', 'Free kick accuracy',
   'GK diving', 'GK handling', 'GK kicking', 'GK positioning',
   'GK reflexes', 'Heading accuracy', 'Interceptions', 'Jumping', 'Long passing', 'Long shots',
	'Marking', 'Penalties', 'Positioning', 'Reactions', 'Short passing', 'Shot power', 
   'Sliding tackle', 'Sprint speed',	'Stamina', 'Standing tackle', 'Strength', 'Vision', 'Volleys']

for attr in attrs:
    if is_string_dtype(df[attr]):
        df[attr] = df[attr].apply(lambda x: eval(x))
            
for attr in attrs:
    df[attr] = pd.to_numeric(df[attr])


In [27]:
def convertToFloat(df_col):
    result = []
    units = {"K":1000,"M":1000000}
    for val in df_col:
            try:
                result.append( float(val) )  
            except ValueError:
                unit=val[-1]                 
                val = float( val[:-1] )        
                result.append( val * units[unit] )
    return result

In [28]:
df["Value"] = df["Value"].str.split('€').str.get(1)
df["Value"]=convertToFloat(df["Value"])
df["Wage"] = df["Wage"].str.split('€').str.get(1)
df["Wage"]=convertToFloat(df["Wage"])

In [31]:
# A : Assign continent
df['Continent'] = 'None'
df2 = pd.read_csv('Datasets/country_continent.csv')
df2.head()

,Country,Continent
0,Afghanistan,Asia
1,Albania,Europe
2,Algeria,Africa
3,Angola,Africa
4,Antigua & Barbuda,North America


In [41]:
# Create a dictionary {Country: Continent}
newdict = {}
for value in list(range(len(df2))):
    newdict[df2.iloc[value,0]] = df2.iloc[value,1]

#assign continent to each player.
for each in list(range(len(df))):
    df['Continent'][each] = newdict[df['Nationality'][each]]   

In [38]:
df.head()

,Name,Age,Nationality,Overall,Potential,Club,Value,Wage,Height_cm,Weight_kg,...,RCB,RCM,RDM,RF,RM,RS,RW,RWB,ST,Continent
0,Cristiano Ronaldo,32,Portugal,94,94,Real Madrid CF,95500000.0,565000.0,185,80,...,53.000000,82.000000,62.000000,91.000000,89.000000,92.00000,91.000000,66.000000,92.00000,Europe
1,L. Messi,30,Argentina,93,93,FC Barcelona,105000000.0,565000.0,170,72,...,45.000000,84.000000,59.000000,92.000000,90.000000,88.00000,91.000000,62.000000,88.00000,South America
2,Neymar,25,Brazil,92,94,Paris Saint-Germain,123000000.0,280000.0,175,68,...,46.000000,79.000000,59.000000,88.000000,87.000000,84.00000,89.000000,64.000000,84.00000,South America
3,L. Suarez,30,Uruguay,92,92,FC Barcelona,97000000.0,510000.0,182,86,...,58.000000,80.000000,65.000000,88.000000,85.000000,88.00000,87.000000,68.000000,88.00000,South America
4,M. Neuer,31,Germany,92,92,FC Bayern Munich,61000000.0,230000.0,193,92,...,55.550464,58.506833,56.865283,59.030028,60.057736,58.20405,59.359265,57.698721,58.20405,Europe


In [66]:
attack = [ 'ST', 'CF', 'RF', 'LF', 'RW', 'LW', 'RS', 'LS' ] 
midfield = [ 'CAM', 'RAM', 'LAM', 'CM', 'RCM', 'LCM', 'LM', 'RM', 'CDM', 'RDM', 'LDM' ]
defense = [ 'CB', 'RB', 'LB', 'RCB', 'LCB', 'RWB', 'LWB' ]

def fieldPositions(atk,mid,defn,df,i):
    if atk == 0 and mid == 0 and defn == 0:
        return 'GK'
    elif atk>mid and atk>defn:
        return 'Attack'
    elif mid>=atk and mid>= defn:
        return 'Midfield'
    elif defn>mid and defn>atk:
        return 'Defense'
    elif atk == mid and mid > defn:
        for pos in attack:
            if df['Preferred Positions'][i] == pos:
                return 'Attack'
        return 'Midfield'
    elif defn == mid and mid > atk:
        for pos in defense:
            if df['Preferred Positions'][i] == pos:
                return 'Defense'
        return 'Midfield'
    elif atk == defn and atk > mid:
        for pos in attack:
            if df['Preferred Positions'][i] == pos:
                return 'Attack'
        return 'Defense'
    elif atk != 0 and mid != 0 and defn != 0 and atk == mid and atk == defn and mid == defn :
        return 'Midfield'    
    else:
        return 'Unknown'

In [67]:
# assign fieldPosition
df['FieldPositions'] = 'None'
df['Atk']=0
df['Mid']=0
df['Def']=0
    
for i in list(range(0,len(df))):
    atk=df[attack].iloc[i].mean()
    mid=df[midfield].iloc[i].mean()
    defn=df[defense].iloc[i].mean()
    df['Atk'][i]=atk
    df['Mid'][i]=mid
    df['Def'][i]=defn 
    df['FieldPositions'][i] = fieldPositions(atk,mid,defn,df,i)

In [69]:
df["Preferred Positions"] = df["Preferred Positions"].str.split()

In [70]:
df["Preferred Positions"].head()

0    [ST, LW]
1        [RW]
2        [LW]
3        [ST]
4        [GK]
Name: Preferred Positions, dtype: object

In [71]:
df['Position'] = "unknown"

for i in list(range(len(df))):
    positions_values = []
    if len(df["Preferred Positions"][i])>1:
        for pos in df["Preferred Positions"][i]:
            positions_values.append(df[pos][i])
        df['Position'][i] = df["Preferred Positions"][i][ positions_values.index(max(positions_values))]
    else:
        df['Position'][i] = df["Preferred Positions"][i][0]

In [75]:
# Step 8 : Write clean data to new csv file and Load it in to MySQL
df.to_csv('Datasets/clean_fifa18.csv', index=False)

In [79]:
from sqlalchemy import create_engine
engine = create_engine('mysql+mysqlconnector://root:NehalBandal@84@localhost:3306/fifa18', echo=False)

In [80]:
personal = ['ID','Name','Age','Nationality','Continent','Club','Wage','Value']
df_personal=pd.DataFrame(df,columns=personal)
df_personal.to_sql(name='player_personal_info', con=engine, if_exists = 'append', index=False)

In [82]:
physical = ['ID','Height_cm', 'Weight_kg', 'Acceleration', 'Aggression', 'Agility',
       'Balance', 'Ball control', 'Composure', 'Crossing', 'Curve',
       'Dribbling', 'Finishing', 'Free kick accuracy', 'GK diving',
       'GK handling', 'GK kicking', 'GK positioning', 'GK reflexes',
       'Heading accuracy', 'Interceptions', 'Jumping', 'Long passing',
       'Long shots', 'Marking', 'Penalties', 'Positioning', 'Reactions',
       'Short passing', 'Shot power', 'Sliding tackle', 'Sprint speed',
       'Stamina', 'Standing tackle', 'Strength', 'Vision', 'Volleys']
df_physical=pd.DataFrame(df,columns=physical)
df_physical.to_sql(name='player_physical_attrs', con=engine, if_exists = 'append', index=False)

In [83]:
positional = ['ID','CAM', 'CB', 'CDM', 'CF', 'CM', 'ID', 'LAM', 'LB', 'LCB', 'LCM', 'LDM',
       'LF', 'LM', 'LS', 'LW', 'LWB', 'RAM', 'RB',
       'RCB', 'RCM', 'RDM', 'RF', 'RM', 'RS', 'RW', 'RWB', 'ST']
df_positional=pd.DataFrame(df,columns=positional)
df_positional.to_sql(name='player_positional_attrs', con=engine, if_exists = 'append', index=False)

In [84]:
abstract = ['ID', 'Atk', 'Mid', 'Def', 'Overall', 'Potential',  'FieldPositions', 'Position']
df_abstract=pd.DataFrame(df,columns=abstract)
df_abstract.to_sql(name='player_abstract_info', con=engine, if_exists = 'append', index=False)